# Practical example. Audiobooks

## Create the machine learning algorithm, L2 Regularization & Dropout

### Import the relevant libraries

In [1]:
import io
import itertools

import numpy as np
import sklearn.metrics

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import matplotlib.pyplot as plt

### Data

In [2]:
npz = np.load('Audiobooks_data_train.npz')

train_inputs = npz['inputs'].astype(np.float)
train_targets = npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

### Model
Outline, optimizers, loss, early stopping and training

In [3]:

max_epochs = 20
batch_size = 64

input_size = 10
output_size = 2

In [4]:

HP_LAMBDA_REG = hp.HParam('lambda', hp.Discrete([0.0, 1e-5, 5e-5, 1e-4, 5e-4, 1e-3, 5e-3, 1e-2, 5e-2, 0.1]))
HP_DROPOUT = hp.HParam('dropout', hp.Discrete([0.0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]))

with tf.summary.create_file_writer(r'Logs/Model 2/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams = [HP_LAMBDA_REG, HP_DROPOUT],
        metrics = [hp.Metric('accuracy', display_name = 'Accuracy')]
    )


In [5]:
# Wrapping our model and training in a function
def train_test_model(hparams, session_num):
    
    # Outlining the model/architecture of our CNN
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(128, 'relu',
                              kernel_regularizer=tf.keras.regularizers.l2(hparams[HP_LAMBDA_REG])),
        
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        
        tf.keras.layers.Dense(128, 'relu',
                              kernel_regularizer=tf.keras.regularizers.l2(hparams[HP_LAMBDA_REG])),
        
        tf.keras.layers.Dropout(hparams[HP_DROPOUT]),
        
        tf.keras.layers.Dense(output_size, activation='softmax')
    ])
    
    # Defining the loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(name='sparse_categorical_crossentropy')
    
    # Compiling the model
    model.compile(optimizer='adam', loss=loss_fn, metrics=['sparse_categorical_crossentropy', 'accuracy'])
    
    # Defining the logging directory
    log_dir = r'Logs/Model 2/fit/' + 'run-{}'.format(session_num)

    
    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
          cm (array, shape = [n, n]): a confusion matrix of integer classes
          class_names (array, shape = [n]): String names of the integer classes
        """
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        return figure
    
    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""
        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        # Closing the figure prevents it from being displayed directly inside
        # the notebook.
        plt.close(figure)
        buf.seek(0)
        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        # Add the batch dimension
        image = tf.expand_dims(image, 0)
        return image
    
    
    # Defining a file writer for Confusion Matrix logging purposes
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')     
    
    
    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = model.predict(validation_inputs)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(validation_targets, test_pred)
        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=["won't buy", "will buy"])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
    
    
    
    # Define the Tensorboard and Confusion Matrix callbacks.
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

    
    # Defining early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'val_sparse_categorical_crossentropy', # We should supply the unregularized the loss
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0, 
        restore_best_weights = True
    )
    
    # Training the model
    model.fit(
        train_inputs,
        train_targets,
        epochs = max_epochs,
        batch_size = batch_size,
        callbacks = [tensorboard_callback, cm_callback, early_stopping],
        validation_data = (validation_inputs, validation_targets),
        verbose = 2
    )
    
    
    # Evaluating the model's performance on the validation set
    # Note that we have three metrics 'see sparse_categorical_crossentropy'
    _, __, accuracy = model.evaluate(validation_inputs, validation_targets)
    
    # Saving the current model for future reference
    model.save(r'saved_models\Model 2\Run-{}'.format(session_num))
    
    return accuracy


In [6]:
# Creating a function to log the resuls
def run(log_dir, hparams, session_num):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar('accuracy', accuracy, step=1)


In [7]:
session_num = 1

for lambda_reg in HP_LAMBDA_REG.domain.values:
    for dropout in HP_DROPOUT.domain.values:
        
        hparams = {
            HP_LAMBDA_REG: lambda_reg,
            HP_DROPOUT: dropout
        }

        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('Logs/Model 2/hparam_tuning/' + run_name, hparams, session_num)

        session_num += 1


--- Starting trial: run-1
{'lambda': 0.0, 'dropout': 0.0}
Epoch 1/20
56/56 - 1s - loss: 0.3970 - sparse_categorical_crossentropy: 0.3970 - accuracy: 0.8583 - val_loss: 0.2862 - val_sparse_categorical_crossentropy: 0.2862 - val_accuracy: 0.8859
Epoch 2/20
56/56 - 1s - loss: 0.2947 - sparse_categorical_crossentropy: 0.2947 - accuracy: 0.8913 - val_loss: 0.2658 - val_sparse_categorical_crossentropy: 0.2658 - val_accuracy: 0.8971
Epoch 3/20
56/56 - 0s - loss: 0.2744 - sparse_categorical_crossentropy: 0.2744 - accuracy: 0.8952 - val_loss: 0.2601 - val_sparse_categorical_crossentropy: 0.2601 - val_accuracy: 0.8993
Epoch 4/20
56/56 - 0s - loss: 0.2658 - sparse_categorical_crossentropy: 0.2658 - accuracy: 0.8997 - val_loss: 0.2552 - val_sparse_categorical_crossentropy: 0.2552 - val_accuracy: 0.9060
Epoch 5/20
56/56 - 1s - loss: 0.2584 - sparse_categorical_crossentropy: 0.2584 - accuracy: 0.9030 - val_loss: 0.2589 - val_sparse_categorical_crossentropy: 0.2589 - val_accuracy: 0.9038
Epoch 6/20
5

Epoch 4/20
56/56 - 1s - loss: 0.3133 - sparse_categorical_crossentropy: 0.3133 - accuracy: 0.8846 - val_loss: 0.2624 - val_sparse_categorical_crossentropy: 0.2624 - val_accuracy: 0.8971
Epoch 5/20
56/56 - 1s - loss: 0.3020 - sparse_categorical_crossentropy: 0.3020 - accuracy: 0.8871 - val_loss: 0.2607 - val_sparse_categorical_crossentropy: 0.2607 - val_accuracy: 0.8949
Epoch 6/20
56/56 - 0s - loss: 0.2930 - sparse_categorical_crossentropy: 0.2930 - accuracy: 0.8933 - val_loss: 0.2606 - val_sparse_categorical_crossentropy: 0.2606 - val_accuracy: 0.8971
Epoch 7/20
56/56 - 1s - loss: 0.2876 - sparse_categorical_crossentropy: 0.2876 - accuracy: 0.8938 - val_loss: 0.2498 - val_sparse_categorical_crossentropy: 0.2498 - val_accuracy: 0.9038
Epoch 8/20
56/56 - 1s - loss: 0.2830 - sparse_categorical_crossentropy: 0.2830 - accuracy: 0.8941 - val_loss: 0.2499 - val_sparse_categorical_crossentropy: 0.2499 - val_accuracy: 0.9038
Epoch 9/20
56/56 - 1s - loss: 0.2802 - sparse_categorical_crossentropy

Epoch 17/20
56/56 - 1s - loss: 0.2806 - sparse_categorical_crossentropy: 0.2806 - accuracy: 0.8989 - val_loss: 0.2433 - val_sparse_categorical_crossentropy: 0.2433 - val_accuracy: 0.9083
Epoch 18/20
56/56 - 0s - loss: 0.2756 - sparse_categorical_crossentropy: 0.2756 - accuracy: 0.8961 - val_loss: 0.2413 - val_sparse_categorical_crossentropy: 0.2413 - val_accuracy: 0.9083
Epoch 19/20
56/56 - 0s - loss: 0.2720 - sparse_categorical_crossentropy: 0.2720 - accuracy: 0.9005 - val_loss: 0.2372 - val_sparse_categorical_crossentropy: 0.2372 - val_accuracy: 0.9083
Epoch 20/20
56/56 - 0s - loss: 0.2744 - sparse_categorical_crossentropy: 0.2744 - accuracy: 0.8986 - val_loss: 0.2391 - val_sparse_categorical_crossentropy: 0.2391 - val_accuracy: 0.9083
14/14 [==============================] - 0s 2ms/step - loss: 0.2391 - sparse_categorical_crossentropy: 0.2391 - accuracy: 0.9083
INFO:tensorflow:Assets written to: saved_models\Model 2\Run-7\assets
--- Starting trial: run-8
{'lambda': 0.0, 'dropout': 0

Epoch 19/20
56/56 - 0s - loss: 0.3135 - sparse_categorical_crossentropy: 0.3135 - accuracy: 0.8832 - val_loss: 0.2559 - val_sparse_categorical_crossentropy: 0.2559 - val_accuracy: 0.9060
Epoch 20/20
56/56 - 0s - loss: 0.3147 - sparse_categorical_crossentropy: 0.3147 - accuracy: 0.8866 - val_loss: 0.2541 - val_sparse_categorical_crossentropy: 0.2541 - val_accuracy: 0.9060
14/14 [==============================] - 0s 2ms/step - loss: 0.2541 - sparse_categorical_crossentropy: 0.2541 - accuracy: 0.9060
INFO:tensorflow:Assets written to: saved_models\Model 2\Run-9\assets
--- Starting trial: run-10
{'lambda': 0.0, 'dropout': 0.9}
Epoch 1/20
56/56 - 1s - loss: 1.1126 - sparse_categorical_crossentropy: 1.1126 - accuracy: 0.5166 - val_loss: 0.6046 - val_sparse_categorical_crossentropy: 0.6046 - val_accuracy: 0.8233
Epoch 2/20
56/56 - 1s - loss: 0.8943 - sparse_categorical_crossentropy: 0.8943 - accuracy: 0.5510 - val_loss: 0.5763 - val_sparse_categorical_crossentropy: 0.5763 - val_accuracy: 0.87

Epoch 2/20
56/56 - 1s - loss: 0.3075 - sparse_categorical_crossentropy: 0.3060 - accuracy: 0.8857 - val_loss: 0.2735 - val_sparse_categorical_crossentropy: 0.2720 - val_accuracy: 0.9016
Epoch 3/20
56/56 - 0s - loss: 0.3030 - sparse_categorical_crossentropy: 0.3015 - accuracy: 0.8874 - val_loss: 0.2591 - val_sparse_categorical_crossentropy: 0.2576 - val_accuracy: 0.8993
Epoch 4/20
56/56 - 0s - loss: 0.2873 - sparse_categorical_crossentropy: 0.2857 - accuracy: 0.8949 - val_loss: 0.2779 - val_sparse_categorical_crossentropy: 0.2764 - val_accuracy: 0.8993
Epoch 5/20
56/56 - 1s - loss: 0.2852 - sparse_categorical_crossentropy: 0.2837 - accuracy: 0.8896 - val_loss: 0.2514 - val_sparse_categorical_crossentropy: 0.2498 - val_accuracy: 0.9038
Epoch 6/20
56/56 - 1s - loss: 0.2778 - sparse_categorical_crossentropy: 0.2762 - accuracy: 0.8969 - val_loss: 0.2465 - val_sparse_categorical_crossentropy: 0.2449 - val_accuracy: 0.9060
Epoch 7/20
56/56 - 0s - loss: 0.2651 - sparse_categorical_crossentropy

...

Epoch 3/20
56/56 - 1s - loss: 0.9851 - sparse_categorical_crossentropy: 0.4150 - accuracy: 0.8611 - val_loss: 0.6954 - val_sparse_categorical_crossentropy: 0.3639 - val_accuracy: 0.8747
Epoch 4/20
56/56 - 1s - loss: 0.6108 - sparse_categorical_crossentropy: 0.3804 - accuracy: 0.8695 - val_loss: 0.5064 - val_sparse_categorical_crossentropy: 0.3477 - val_accuracy: 0.8770
Epoch 5/20
56/56 - 1s - loss: 0.4995 - sparse_categorical_crossentropy: 0.3706 - accuracy: 0.8712 - val_loss: 0.4473 - val_sparse_categorical_crossentropy: 0.3393 - val_accuracy: 0.8792
Epoch 6/20
56/56 - 1s - loss: 0.4629 - sparse_categorical_crossentropy: 0.3632 - accuracy: 0.8745 - val_loss: 0.4220 - val_sparse_categorical_crossentropy: 0.3329 - val_accuracy: 0.8792
Epoch 7/20
56/56 - 1s - loss: 0.4504 - sparse_categorical_crossentropy: 0.3626 - accuracy: 0.8712 - val_loss: 0.4111 - val_sparse_categorical_crossentropy: 0.3259 - val_accuracy: 0.8747
Epoch 8/20
56/56 - 1s - loss: 0.4421 - sparse_categorical_crossentropy

Epoch 10/20
56/56 - 1s - loss: 0.4817 - sparse_categorical_crossentropy: 0.3849 - accuracy: 0.8609 - val_loss: 0.4311 - val_sparse_categorical_crossentropy: 0.3355 - val_accuracy: 0.8747
Epoch 11/20
56/56 - 1s - loss: 0.4743 - sparse_categorical_crossentropy: 0.3792 - accuracy: 0.8678 - val_loss: 0.4242 - val_sparse_categorical_crossentropy: 0.3322 - val_accuracy: 0.8770
Epoch 12/20
56/56 - 1s - loss: 0.4695 - sparse_categorical_crossentropy: 0.3759 - accuracy: 0.8653 - val_loss: 0.4251 - val_sparse_categorical_crossentropy: 0.3368 - val_accuracy: 0.8770
Epoch 13/20
56/56 - 1s - loss: 0.4664 - sparse_categorical_crossentropy: 0.3751 - accuracy: 0.8620 - val_loss: 0.4158 - val_sparse_categorical_crossentropy: 0.3241 - val_accuracy: 0.8792
Epoch 14/20
56/56 - 1s - loss: 0.4611 - sparse_categorical_crossentropy: 0.3705 - accuracy: 0.8690 - val_loss: 0.4137 - val_sparse_categorical_crossentropy: 0.3221 - val_accuracy: 0.8747
Epoch 15/20
56/56 - 1s - loss: 0.4605 - sparse_categorical_crosse

Epoch 2/20
56/56 - 1s - loss: 3.2805 - sparse_categorical_crossentropy: 0.5039 - accuracy: 0.8522 - val_loss: 1.8232 - val_sparse_categorical_crossentropy: 0.4352 - val_accuracy: 0.8725
Epoch 3/20
56/56 - 1s - loss: 1.2355 - sparse_categorical_crossentropy: 0.4257 - accuracy: 0.8653 - val_loss: 0.8165 - val_sparse_categorical_crossentropy: 0.3922 - val_accuracy: 0.8747
Epoch 4/20
56/56 - 1s - loss: 0.6745 - sparse_categorical_crossentropy: 0.3978 - accuracy: 0.8681 - val_loss: 0.5519 - val_sparse_categorical_crossentropy: 0.3722 - val_accuracy: 0.8747
Epoch 5/20
56/56 - 1s - loss: 0.5265 - sparse_categorical_crossentropy: 0.3821 - accuracy: 0.8748 - val_loss: 0.4832 - val_sparse_categorical_crossentropy: 0.3584 - val_accuracy: 0.8792
Epoch 6/20
56/56 - 1s - loss: 0.4874 - sparse_categorical_crossentropy: 0.3733 - accuracy: 0.8704 - val_loss: 0.4640 - val_sparse_categorical_crossentropy: 0.3538 - val_accuracy: 0.8747
Epoch 7/20
56/56 - 1s - loss: 0.4772 - sparse_categorical_crossentropy

Epoch 10/20
56/56 - 1s - loss: 0.4856 - sparse_categorical_crossentropy: 0.3813 - accuracy: 0.8614 - val_loss: 0.4471 - val_sparse_categorical_crossentropy: 0.3480 - val_accuracy: 0.8747
Epoch 11/20
56/56 - 1s - loss: 0.4731 - sparse_categorical_crossentropy: 0.3732 - accuracy: 0.8690 - val_loss: 0.4422 - val_sparse_categorical_crossentropy: 0.3462 - val_accuracy: 0.8792
14/14 [==============================] - 0s 1ms/step - loss: 0.4484 - sparse_categorical_crossentropy: 0.3426 - accuracy: 0.8747
INFO:tensorflow:Assets written to: saved_models\Model 2\Run-97\assets
--- Starting trial: run-98
{'lambda': 0.1, 'dropout': 0.7}
Epoch 1/20
56/56 - 2s - loss: 9.9272 - sparse_categorical_crossentropy: 0.7143 - accuracy: 0.5719 - val_loss: 5.5684 - val_sparse_categorical_crossentropy: 0.5942 - val_accuracy: 0.8568
Epoch 2/20
56/56 - 1s - loss: 3.5509 - sparse_categorical_crossentropy: 0.6026 - accuracy: 0.7343 - val_loss: 2.0573 - val_sparse_categorical_crossentropy: 0.5531 - val_accuracy: 0.8

Epoch 2/20
56/56 - 1s - loss: 4.6695 - sparse_categorical_crossentropy: 0.7652 - accuracy: 0.5541 - val_loss: 3.0327 - val_sparse_categorical_crossentropy: 0.6309 - val_accuracy: 0.7494
Epoch 3/20
56/56 - 1s - loss: 2.3048 - sparse_categorical_crossentropy: 0.6748 - accuracy: 0.5907 - val_loss: 1.6815 - val_sparse_categorical_crossentropy: 0.6398 - val_accuracy: 0.7606
14/14 [==============================] - 0s 1ms/step - loss: 6.4868 - sparse_categorical_crossentropy: 0.6169 - accuracy: 0.8389
INFO:tensorflow:Assets written to: saved_models\Model 2\Run-100\assets


In [8]:
%load_ext tensorboard
%tensorboard --logdir "Logs/Model 2/hparam_tuning"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 8096), started 0:03:26 ago. (Use '!kill 8096' to kill it.)

In [16]:
%load_ext tensorboard
%tensorboard --logdir "Logs/Model 2/fit"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 8776), started 0:01:04 ago. (Use '!kill 8776' to kill it.)

## Test the model

In [10]:
# Loading a model to evaluate on the test set
model = tf.keras.models.load_model(r"saved_models\Model 2\Run-1")

In [11]:
test_pred = np.argmax(model.predict(test_inputs), axis=1)
test_accuracy = (test_targets == test_pred).sum() / test_targets.shape[0]

In [12]:
print('\nTest accuracy: {0:.2f}%'.format(test_accuracy * 100.))


Test accuracy: 90.40%
